In [94]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from folium import LayerControl
import geopandas as gpd

In [95]:
data = pd.read_excel("E://files_pandas//sisver//Unidades.xlsx")
data = data[['NOMBRE DE LA UNIDAD', 'LATITUD', 'LONGITUD']]
data['LATITUD'] = pd.to_numeric(data['LATITUD'], errors='coerce')
data['LONGITUD'] = pd.to_numeric(data['LONGITUD'], errors='coerce')
data = data.dropna(subset=['LATITUD', 'LONGITUD'])

In [ ]:


try:
    gdf = gpd.read_file("E://files_pandas//sisver//muni_2018gw//muni_2018gw.shp", encoding='utf-8')
except UnicodeDecodeError:
    gdf = gpd.read_file("E://files_pandas//sisver//muni_2018gw//muni_2018gw.shp", encoding='latin-1')

# SELECCIONAR SOLO HIDALGO
hgo = gdf[gdf['NOM_ENT'] == 'Hidalgo']

# Crear el mapa centrado en la ubicación media de los datos
mapa = folium.Map(location=[data['LATITUD'].mean(), data['LONGITUD'].mean()], zoom_start=10)

# Añadir capa de clúster
marker_cluster = MarkerCluster().add_to(mapa)

# Añadir puntos y círculos al mapa dentro de la capa de clúster
for _, row in data.iterrows():
    folium.Marker(
        location=[row['LATITUD'], row['LONGITUD']],
        popup=row['NOMBRE DE LA UNIDAD']
    ).add_to(marker_cluster)
    
    folium.Circle(
        location=[row['LATITUD'], row['LONGITUD']],
        radius=500,  # en metros, puedes ajustar según lo necesites
        color='#ADB5BD',
        fill=True,
        fill_color='#DEE2E6'
    ).add_to(mapa)

# Listas de municipios para cada jurisdicción
municipios_dict = {
    'Jurisdicción Tula': ['Ajacuba','Atitalaquia','Atotonilco de Tula','Tula de Allende','Tepeji del Río de Ocampo','Tepetitlán','Tetepango','Tezontepec de Aldama','Tlahuelilpan','Tlaxcoapan'],
    'Jurisdicción Tulancingo': ['Acatlán','Acaxochitlán','Agua Blanca de Iturbide','Cuautepec de Hinojosa','Huehuetla','Metepec','San Bartolo Tutotepec','Santiago Tulantepec de Lugo Guerrero','Singuilucan','Tenango de Doria','Tulancingo de Bravo'],
    'Jurisdicción Pachuca': ['Pachuca de Soto','San Agustín Tlaxiaca','Zapotlán de Juárez'],
    'Jurisdicción Huejutla': ['Atlapexco','Calnali','Huautla','Huazalingo','Huejutla de Reyes','Jaltocán','Lolotla','San Felipe Orizatlán','Tlanchinol','Xochiatipan','Yahualica'],
    'Jurisdicción Mineral': ['Epazoyucan','Mineral de la Reforma','Mineral del Chico','Omitlán de Juárez','Mineral del Monte','Huasca de Ocampo'],
    'Jurisdicción Tizayuca': ['Villa de Tezontepec','Tolcayuca','Tizayuca','Zempoala'],
    'Jurisdicción Actopan': ['Actopan','El Arenal','Francisco I. Madero','Mixquiahuala de Juárez','Progreso de Obregón','San Salvador','Santiago de Anaya'],
    'Jurisdicción Ixmiquilpan': ['Alfajayucan','Cardonal','Chilcuautla','Ixmiquilpan','Nicolás Flores','Tasquillo','Zimapán'],
    'Jurisdicción Molango': ['Atotonilco el Grande','Eloxochitlán','Juárez Hidalgo','Metztitlán','San Agustín Metzquititlán','Molango de Escamilla','Tepehuacán de Guerrero','Tianguistengo','Tlahuiltepa','Xochicoatlán','Zacualtipán de Ángeles'],
    'Jurisdicción Apan': ['Almoloya', 'Apan','Emiliano Zapata','Tepeapulco','Tlanalapa'],
    'Jurisdicción Huichapan': ['Chapantongo','Huichapan','Nopala de Villagrán','Tecozautla'],
    'Jurisdicción Jacala': ['Chapulhuacán','Jacala de Ledezma','La Misión','Pacula','Pisaflores']
}

# Colores para cada jurisdicción
colores = {
    'Jurisdicción Tula': '#52796F', 'Jurisdicción Tulancingo': '#3f37c9', 'Jurisdicción Pachuca': '#4cc9f0', 'Jurisdicción Huejutla': '#ffd60a', 'Jurisdicción Mineral': '#343A40',
    'Jurisdicción Tizayuca': '#354F52', 'Jurisdicción Actopan': '#f28482', 'Jurisdicción Ixmiquilpan': '#d80032', 'Jurisdicción Molango': '#db7c26', 'Jurisdicción Apan': '#274c77',
    'Jurisdicción Huichapan': '#eddea4', 'Jurisdicción Jacala': '#00afb9'
}

# Agrupar municipios y añadir capas de cada jurisdicción
for nombre_jurisdiccion, municipios in municipios_dict.items():
    grupo = folium.FeatureGroup(name=nombre_jurisdiccion)
    for municipio in municipios:
        folium.GeoJson(
            hgo[hgo['NOM_MUN'] == municipio],
            style_function=lambda feature, color=colores[nombre_jurisdiccion]: {
                'fillColor': color,
                'color': color,
                'weight': 2,
                'fillOpacity': 0.5,
                'opacity': 0.7,
                'lineCap': 'round',
                'lineJoin': 'round'
            }
        ).add_to(grupo)
    grupo.add_to(mapa)

# Añadir controles de capas
folium.LayerControl().add_to(mapa)

title_html = ''' <h3 align="center" style="font-size:20px"><b>Mapa de Unidades de salud Hidalgo</b></h3> '''
mapa.get_root().html.add_child(folium.Element(title_html))
# Guardar el mapa en un archivo HTML
mapa.save('E://files_pandas//sisver//mapa_localidades.html')

# Mostrar mensaje de éxito
print("El mapa se ha guardado exitosamente en 'mapa_localidades.html'")


NameError: name 'data' is not defined